This tutorial explains how our script converts detected issues into a '.bcfzip' file. The goal is to export the errors, already found in the script, into BCF issues that can be opened in tools like blender. 

Each BCF issue will contain
- a title and description
- highlighted IFC elements
- a camera viewpoint focusing on the issue
- an automatic timestamped comment

In [ ]:
from rich.console import Console
import ifcopenshell
from ifcopenshell.util.file import IfcHeaderExtractor
from bcf.v3.bcfxml import BcfXml
import bcf.v3.visinfo
import bcf.v3.model
from .functions import get_element_bbox
import datetime
import uuid

Explanation of imports
- 'BcfXmL': creates and writes 'bcfzip' files
- 'bcf.v3.visinfo': stores viewpoints and selected elements
- 'bcf.v3.model': defines BCF objects such as comments
- 'IfcHeaderExtractor': read project name from IFC file to label the BCF project
- 'get_element_bbox': helper that finds the 3D bounding box of the elements for camera position

In [ ]:
def iso_now() -> str:
    return datetime.datetime.now(datetime.timezone.utc).replace(microsecond=0).isoformat() + "Z"

Every BCF needs a timestamp, which is created by this function. 
The current time is returned in the ISO format required by BCF comments, which is used for each BCF issue.

In [ ]:
def cameraSetup(element: ifcopenshell.entity_instance, 
                ifc_file: ifcopenshell.file) -> tuple[list[float], list[float], list[float]]:

    if isinstance(element, list):
        element = element[0]
    
    bbox = get_element_bbox(element)
    center = [(bbox['min'][i] + bbox['max'][i]) / 2 for i in range(3)]

    camera_view_point = [float(bbox['max'][0]*1.04), float(bbox['max'][1]*1.04), float(bbox['max'][2]*1.04)]
    
    camera_direction = [float(center[0] - camera_view_point[0]),
                       float(center[1] - camera_view_point[1]),
                       float(center[2] - camera_view_point[2])]
    
    camera_up_vector = [0.0, 0.0, 1.0]

    return camera_view_point, camera_direction, camera_up_vector

This function positions a camera so the BCF viewer looks directly at the element with an issue. 

Steps:
1. Gets the 3D bounding box of the element
2. Calculates the geometric center
3. Places the camera outside the element
4. Points the camera towards the center of the element
5. Sets z-axis as "up"

In [ ]:
def add_issue(bcf_obj: BcfXml, title: str, message: str, 
              author: str, element: ifcopenshell.entity_instance, ifc_file: ifcopenshell.file,
              bcf_path: str, bcf_zip: bcf.v3.visinfo.ZipFileInterface) -> None:
    th = bcf_obj.add_topic(
        title,
        message,
        author,
        "Parameter Validation",
    )
    
    if isinstance(element, list):
        # inspect(element) # FOR DEBUGGING
        visinfo_handler = th.add_viewpoint(element[0])
        visinfo_handler.set_selected_elements(element)
    else:
        # inspect(element)
        visinfo_handler = th.add_viewpoint(element)
        visinfo_handler.set_selected_elements([element])

    camera_view_point, camera_direction, camera_up_vector = cameraSetup(element=element, ifc_file=ifc_file)
    visinfo_handler.visualization_info.perspective_camera = bcf.v3.visinfo.build_camera_from_vectors(camera_position=camera_view_point, camera_dir=camera_direction, camera_up=camera_up_vector)
    # inspect(visinfo_handler) # FOR DEBUGGING  

    th.comments = [bcf.v3.model.Comment(
        guid=str(uuid.uuid4()),
        date=iso_now(),
        author=author,
        comment=message,
        viewpoint=bcf.v3.model.CommentViewpoint(guid=visinfo_handler.guid),
    )]

This function creates a single BCF issue that includes:

- a topic
- a viewpoint zoomed in on the element
- selected elements highlighted
- a message of the issue

It supports:
- a single element
- a list of elements

The function does not loop but creates one BCF per issue

In [ ]:
def generate_bcf_from_errors(
    console: Console,
    ifc_file: ifcopenshell.file,
    ifc_file_path: str,
    misplacedElements: list,
    missingAHUsystems: dict,
    unassignedTerminals: dict,
    output_bcf: str = "hvac_issues.bcfzip") -> None:

    console.print("🔧 Opening IFC file...")
    extractor = IfcHeaderExtractor(ifc_file_path)
    header_info = extractor.extract()
    console.print("📦 Creating new BCF project...")
    bcf_project = BcfXml.create_new(project_name=header_info.get('name'))
    bcf_project.save(filename=output_bcf, keep_open=True)
    bcf_zip = bcf_project._zip_file
    
    # misplaced elements
    console.print("🧱 Adding misplaced element topics...")
    for error_category, elements in misplacedElements.items():
        for guid, e in elements.items():
            title = f"{error_category}: {e.get('elementType', 'Unknown')} representation issue - {guid}"
            desc_lines = [
                f"Element Type: {e.get('elementType', 'Unknown')}",
                f"Issue Category: {error_category}",
            ]

            # Add more context if fields exist
            if "originalLevel" in e and "newLevel" in e:
                desc_lines.append(
                    f"Moved from {e['originalLevel']} (elev {e.get('originalLevelElevation')}) "
                    f"to {e['newLevel']} (elev {e.get('newLevelElevation')})."
                )
            if "minZ" in e and "maxZ" in e:
                desc_lines.append(
                    f"Min Z: {e['minZ']}, Max Z: {e['maxZ']}, Height: {e.get('elementHeight')}"
                )

            desc = "\n".join(desc_lines)

            add_issue(bcf_obj=bcf_project, title=title, message=desc, author="HVAC-Checker", 
                      element=ifc_file.by_id(guid), ifc_file=ifc_file, bcf_path=output_bcf, bcf_zip=bcf_zip)

    # systems missing AHUs
    console.print("💨 Adding missing AHU system topics...")
    for sys_name, info in missingAHUsystems.items():
        title = f"IfcDistributionSystem - {sys_name}: Missing Air Handling Unit"
        desc = (
            f"The distribution system '{sys_name}' contains {info.get('ElementCount', 0)} elements "
            f"but no AHU (IfcUnitaryUnit) was found.\n"
            f"Types: {info.get('ElementTypes', 'Unknown')}."
        )
    
    # uassigned air terminals
    console.print("🌬️ Adding unassigned terminal topics...")
    for flow_dir, element_list in unassignedTerminals.items():
        for element in element_list:
            title = f"Air terminal not placed inside a space - ({element})"
            desc = f"Air terminal {element} - {flow_dir} is not located inside an IfcSpace."
            
            add_issue(bcf_obj=bcf_project, title=title, message=desc, author="HVAC-Checker", 
                      element=ifc_file.by_id(element), ifc_file=ifc_file, bcf_path=output_bcf, bcf_zip=bcf_zip)
    
    # save bcf file
    bcf_project.save(filename=output_bcf)
        console.print(f"✅ BCF file successfully written: {output_bcf}")


This is the function that creates the '.bcfzip' file

1. It gets the project name from the IFC header
2. Creates a new BCF project file
3. Loops through three groups of issues
    - misplaced elements -> one topic per element
    - missing AHU systems -> one topic per system
    - unassigned air terminals -> one topic per terminal
4. For each issue, adds the issue to the BCF project file
5. Saves the BCF file